In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from lac.perception.segmentation import SemanticClasses
from lac.slam.backend import SemanticPointCloud
from lac.mapping.mapper import bin_points_to_grid, nanmedian_filter
from lac.mapping.map_utils import get_geometric_score, get_rocks_score
from lac.mapping.interpolation import interpolate_heights, interpolate_heights_rbf
from lac.utils.plotting import plot_heatmap, plot_surface
from lac.util import load_data

%load_ext autoreload
%autoreload 2

# Load data


In [ ]:
# Load the data logs
# data_path = "/home/shared/data_raw/LAC/runs/double_loop_preset1"
data_path = "../../output/NavAgent/preset6_5loops"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

In [ ]:
ground_truth_map = np.load(Path(data_path) / "Moon_Map_01_6_rep0.dat", allow_pickle=True)
agent_map = np.load(Path(data_path) / "Moon_Map_01_6_rep0_agent.dat", allow_pickle=True)

In [ ]:
agent_map = ground_truth_map.copy()

In [ ]:
point_map = SemanticPointCloud.from_file(Path(data_path) / "semantic_points.npz")

# Height mapping


In [ ]:
ground_points = point_map.points[point_map.labels == SemanticClasses.GROUND.value]
ground_grid = bin_points_to_grid(ground_points, statistic="robust_mean")

In [ ]:
ground_grid = nanmedian_filter(ground_grid, size=3)

In [ ]:
agent_map[:, :, 2] = ground_grid

In [ ]:
plot_surface(agent_map)

In [ ]:
agent_map[:] = interpolate_heights(agent_map)

In [ ]:
plot_surface(agent_map)

In [ ]:
print(f"Geometric score: {get_geometric_score(ground_truth_map, agent_map)}")
print(f"Rocks score: {get_rocks_score(ground_truth_map, agent_map)}")

# Rock mapping
